In [1]:
from fastai import *        # Quick accesss to most common functionality
from fastai.text import *   # Quick accesss to NLP functionality
from fastai.docs import *   # Access to example data provided with fastai

# Text example

An example of creating a language model and then transfering to a classifier.

In [2]:
untar_imdb()
IMDB_PATH

PosixPath('../data/imdb_sample')

Open and view the independent and dependent variables:

In [3]:
df = pd.read_csv(IMDB_PATH/'train.csv', header=None)
df.head()

,0,1
0,0,Un-bleeping-believable! Meg Ryan doesn't even ...
1,1,This is a extremely well-made film. The acting...
2,0,Every once in a long while a movie will come a...
3,1,Name just says it all. I watched this movie wi...
4,0,This movie succeeds at being one of the most u...


In [4]:
classes = read_classes(IMDB_PATH/'classes.txt')
classes[0], classes[1]

('negative', 'positive')

Create a `DataBunch` for each of the language model and the classifier:

In [5]:
data_lm = text_data_from_csv(Path(IMDB_PATH), data_func=lm_data)
data_clas = text_data_from_csv(Path(IMDB_PATH), data_func=classifier_data, vocab=data_lm.train_ds.vocab)

[fast.ai](http://www.fast.ai/) has a pre-trained English model available that we can download.

In [6]:
download_wt103_model()

We'll fine-tune the language model:

In [7]:
learn = RNNLearner.language_model(data_lm, pretrained_fnames=['lstm_wt103', 'itos_wt103'])
learn.unfreeze()
learn.fit(2, slice(1e-4,1e-2))

Total time: 00:11
epoch  train loss  valid loss  accuracy
0      4.876736    4.128457    0.251044  (00:05)
1      4.612240    4.065170    0.257412  (00:05)



Save our language model's encoder:

In [8]:
learn.save_encoder('enc')

Fine tune it to create a classifier:

In [11]:
learn = RNNLearner.classifier(data_clas)
learn.load_encoder('enc')
learn.fit(3, 1e-3)

Total time: 00:14
epoch  train loss  valid loss  accuracy
0      0.690279    0.681847    0.565000  (00:04)
1      0.673246    0.635018    0.695000  (00:04)
2      0.653075    0.607211    0.690000  (00:05)

